In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence = True

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
outdir_csv = debris_prms.mb_binned_fp
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)

In [3]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)
dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])
# dc_shp_subset

rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi_subset

219 glaciers in region 2 are included in this model run: ['00280', '00737', '00914', '01104', '01152', '01158', '01161', '01290', '01291', '01297', '01339', '01397', '01441', '01654', '01665', '01685', '01727', '01811', '01812', '01922', '01923', '02107', '02348', '02360', '02386', '02432', '02526', '02527', '02533', '02550', '02551', '02616', '02636', '02686', '02745', '02747', '02752', '02784', '02857', '02894', '02897', '02947', '02948', '02966', '03099', '03102', '03157', '03520', '03578', '03581'] and more
This study is focusing on 219 glaciers in region [2]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,279,RGI60-02.00280,-122.57300,49.88620,2,2,5.938,1671,2340,1949,14.2,31,4652,0,0,0,20059999,280,02.00280,2.00280
1,736,RGI60-02.00737,-116.52800,50.21440,2,3,5.536,2237,3104,2586,13.6,358,4427,0,0,0,20059999,737,02.00737,2.00737
2,913,RGI60-02.00914,-123.85700,50.31570,2,2,4.372,1056,2233,1486,14.0,5,5531,0,0,0,20059999,914,02.00914,2.00914
3,1103,RGI60-02.01104,-122.60500,50.41950,2,2,3.016,1850,2539,2054,11.1,9,3589,0,0,0,20059999,1104,02.01104,2.01104
4,1151,RGI60-02.01152,-123.89000,50.34100,2,2,12.021,1280,2505,1908,19.4,115,5490,0,0,0,20059999,1152,02.01152,2.01152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,18764,RGI60-02.18765,-121.01578,48.31119,2,4,4.845,1523,2642,2125,23.4,15,3570,0,0,0,19589999,18765,02.18765,2.18765
215,18769,RGI60-02.18770,-121.00437,48.07234,2,4,2.125,1845,2490,2253,27.5,17,1431,0,0,0,19849999,18770,02.18770,2.18770
216,18777,RGI60-02.18778,-121.05735,48.35698,2,4,2.924,1613,2196,1891,12.8,350,3338,0,0,0,19589999,18778,02.18778,2.18778


In [4]:
# np.where(main_glac_rgi_subset.rgino_str == '01.09616')

In [ ]:
# ===== PROCESS EACH GLACIER =====
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str

    if os.path.exists(debris_prms.hd_fp + debris_prms.hd_fn_sample.replace('XXXX',glac_str_noleadzero)) == False:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        gf = create_glacfeat(thick_dir, thick_fn)
        
        # Debris shape layer processing
        dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                          str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
        if os.path.exists(dc_shp_proj_fn) == False:
            dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
            dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
            dc_shp_single = dc_shp_single.reset_index()
            dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
            dc_shp_proj.to_file(dc_shp_proj_fn)
        dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
        dc_shp_lyr = dc_shp_ds.GetLayer()
        
        
        # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
        mb_fullfns = []
        find_mb = True
        dhdt_fn_wglacier = None
        for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
            if find_mb:
                for i in os.listdir(mb_fp):
                    if i.endswith('.tif'):
                        mb_fullfns.append(mb_fp + i)
                tif_count = 0
                while find_mb and tif_count < len(mb_fullfns):
                    dhdt_fn = mb_fullfns[tif_count]
                    if debug:
                        print(tif_count, dhdt_fn.split('/')[-1])
                    
                    # Add the filenames
                    fn_dict = OrderedDict()
                    # DEM
                    z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                    z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                    fn_dict['z1'] = z1_fp + z1_fn
                    # Ice thickness
                    thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                    thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                    fn_dict['ice_thick'] = thick_dir + thick_fn
                    # dh/dt
                    fn_dict['dhdt'] = dhdt_fn
                    # ===== PROCESS THE DATA =====
                    #Expand extent to include buffered region around glacier polygon
                    warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                    #Warp everything to common res/extent/proj
                    z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                    z1_res = np.min([z1_gt[1], -z1_gt[5]])
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'z1' in ds_dict:
                        #This is False over glacier polygon surface, True elsewhere - can be applied directly
                        glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                        gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                        glac_geom_mask_copy = glac_geom_mask.copy()
                        if 'dhdt' in ds_dict:
                            gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                            gf.dhdt.mask = np.ma.mask_or(
                                glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                 np.isnan(gf.dhdt.data))))
                            # Count dhdt pixels
                            dhdt_pixels = len(gf.dhdt.nonzero()[0])
                            if dhdt_pixels / gf.z1.count() * 100 > 75:
                                dhdt_fn_wglacier = dhdt_fn
                                find_mb = False
                                if debug:
                                    print('\n# z1 pixels:', gf.z1.count())
                                    print('# dhdt_pixels:', dhdt_pixels)
                                    var_full2plot = gf.dhdt.copy()
                                    clim = malib.calcperc(var_full2plot, (2,98))
                                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                               close_fig=False)
                    # Loop over layers        
                    tif_count += 1
                    
                    
        # ==== CHECK IF TIF HAS VELOCITY DATA OVER THE GLACIER =====
        vel_fullfns = []
        find_vel = True
        vel_fn_wglacier = None
        if find_vel:
            vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
            tif_count = 0
            while find_vel and tif_count < len(vx_fns):
                vx_fn = vx_fns[tif_count]

                if debug:
                    print(tif_count, vx_fn.split('/')[-1])

                # Add the filenames
                fn_dict = OrderedDict()
                # DEM
                z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                fn_dict['z1'] = z1_fp + z1_fn
                # Ice thickness
                thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                fn_dict['ice_thick'] = thick_dir + thick_fn
                # Velocity
                fn_dict['vx'] = vx_fn
                fn_dict['vy'] = vx_fn.replace('_vx', '_vy')

                # ===== PROCESS THE DATA =====
                #Expand extent to include buffered region around glacier polygon
                warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                #Warp everything to common res/extent/proj
                z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                z1_res = np.min([z1_gt[1], -z1_gt[5]])
                ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                   t_srs=gf.aea_srs, verbose=False, r='cubic')
                ds_dict = dict(zip(fn_dict.keys(), ds_list))
                gf.ds_dict = ds_dict
                if 'z1' in ds_dict:
                    #This is False over glacier polygon surface, True elsewhere - can be applied directly
                    glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                    gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                    glac_geom_mask_copy = glac_geom_mask.copy()
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        gf.vm_mean = gf.vm.mean()
                        if debug:
                            print('mean velocity [m/s]:', gf.vm_mean)
                            
                        # Count vel pixels
                        vel_pixels = len(gf.vm.nonzero()[0])
                        if vel_pixels / gf.z1.count() * 100 > 75:
                            vx_fn_wglacier = vx_fn
                            find_vel = False
                            if debug:
                                print('\n# z1 pixels:', gf.z1.count())
                                print('# dhdt_pixels:', vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                # Loop over layers        
                tif_count += 1

        
        # ===== Add layers =====
        if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, vx_fn=vx_fn_wglacier, 
                          gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, calc_emergence=True, 
                          debug_emergence=False)

            # ===== PLOTS =====
#             plot_layers = True
            plot_layers = False
            if plot_layers:
                # DEM
                var_full2plot = gf.z1.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                # Elevation change
                var_full2plot = gf.dhdt.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                # Velocity
                var_full2plot = gf.vm.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                # Emergence velocity
                if gf.emvel is not None:
                    var_full2plot = gf.emvel.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                # Surface temperature
                if gf.ts is not None:
                    var_full2plot = gf.ts.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)

            # Bin data
            outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
            # Export binned data
            if int(gf.feat_fn.split('.')[0]) < 10:
                outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
            else:
                outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
            outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
            outbins_df.to_csv(outbins_fullfn, index=False)

0 0 RGI60-02.00280
1 1 RGI60-02.00737
2 2 RGI60-02.00914
3 3 RGI60-02.01104
4 4 RGI60-02.01152
5 5 RGI60-02.01158
6 6 RGI60-02.01161
7 7 RGI60-02.01290
8 8 RGI60-02.01291
9 9 RGI60-02.01297
10 10 RGI60-02.01339
11 11 RGI60-02.01397
12 12 RGI60-02.01441
13 13 RGI60-02.01654
14 14 RGI60-02.01665
15 15 RGI60-02.01685
16 16 RGI60-02.01727
17 17 RGI60-02.01811
18 18 RGI60-02.01812
19 19 RGI60-02.01922
20 20 RGI60-02.01923
21 21 RGI60-02.02107
22 22 RGI60-02.02348
23 23 RGI60-02.02360
24 24 RGI60-02.02386
25 25 RGI60-02.02432
26 26 RGI60-02.02526
27 27 RGI60-02.02527
28 28 RGI60-02.02533
29 29 RGI60-02.02550
30 30 RGI60-02.02551
31 31 RGI60-02.02616
32 32 RGI60-02.02636
33 33 RGI60-02.02686
34 34 RGI60-02.02745
35 35 RGI60-02.02747
36 36 RGI60-02.02752
37 37 RGI60-02.02784
38 38 RGI60-02.02857
39 39 RGI60-02.02894
40 40 RGI60-02.02897
41 41 RGI60-02.02947
42 42 RGI60-02.02948
43 43 RGI60-02.02966
44 44 RGI60-02.03099
45 45 RGI60-02.03102
46 46 RGI60-02.03157
47 47 RGI60-02.03520
48 48 RGI60-

In [ ]:
print('\n\nDONE!\n\n')

In [ ]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)